Test


### <a id='toc1_1_1_'></a>[Importing libraries](#toc0_)

In [1]:
import os
from dotenv import load_dotenv
from pprint import pprint

import pandas as pd

import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

import google.generativeai as genai

from IPython.display import Markdown

/Users/wesleymaa/Downloads/LegalTechRAG/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### <a id='toc1_1_2_'></a>[The Gemini API](#toc0_)

In [2]:
load_dotenv()

api_key = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=api_key)

#### <a id='toc1_1_2_2_'></a>[Available embedding models](#toc0_)

In [3]:
for m in genai.list_models():
    if 'embedContent' in m.supported_generation_methods:
        print(m.name)

models/embedding-001


### <a id='toc1_1_4_'></a>[The embedding database](#toc0_)

In [9]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        model = 'models/embedding-001'
        # for better results, try to provide a title for each input if the corpus is covering a lot of domains
        title = ""

        return genai.embed_content(
            model=model,
            content=input,
            task_type="retrieval_document",
            title=title)["embedding"]

In [10]:
chroma_client = chromadb.PersistentClient(path="../database/")
db = chroma_client.get_or_create_collection(
    name="sme_db", embedding_function=GeminiEmbeddingFunction())

In [4]:
def get_relevant_passages(query, db, n_results=5):
    passages = db.query(query_texts=[query], n_results=n_results)[
        'documents'][0]
    return passages

In [15]:
question = "What is continuous presence?"
passages = get_relevant_passages(question, db, n_results=5)

Markdown(passages[2])

the United States through testimony and documentation
For example, a “commuter alien” may have held and used a PRC  for 7 years, but would not be eligible
for naturalization until he or she had actually taken up permanent residence in the United States and
maintained such residence for the required statutory period
USCIS will review all of the relevant records to determine whether the applicant has met the required
period of continuous residence The applicant's testimony will also be considered to determine whether
the applicant met the required period of continuous residence 
[^ 1] See INA 316(a)
[^ 2] See INA 316(a) See Chapter 6, Jurisdiction, Place of Residence, and Early Filing [12 USCIS-PM D6]
[^ 3] See 8 CFR 3165(a)
[^ 4] See Chapter 5, Modifications and Exceptions to Continuous Residence and Physical Presence [12
USCIS-PM D5]
[^ 5] See Part I, Military Members and their Families [12 USCIS-PM I]
[^ 6] See Chapter 4, Physical Presence [12 USCIS-PM D4]